flow of code 
    1-convert json into data structure
    2-helper function to closure of node
    3-transantions table
todo list
    1-coverting json => Done
    2-Get Closure    => Done
    3-Define DFA     => Done
    4-algorithm   

In [8]:
from tokenize import String

In [9]:
import json

# Read the JSON file
with open('B:\\Spring2024\\Compilers\\Assignment\\test.json') as f:
    data = json.load(f)

# Print the data
print(data['startingState'])
startState = data['startingState'];
states = {key: value for key, value in data.items() if key != 'startingState'}
print("States:", states)



S0
States: {'S0': {'isTerminatingState': False, 'A': 'S1', 'B': 'S0', ' ': 'S2', 'E': 'S3'}, 'S1': {'isTerminatingState': False, 'A': 'S1', 'B': 'S1', 'E': 'S0'}, 'S2': {'isTerminatingState': False, 'A': 'S1', 'B': 'S0', ' ': 'S1', 'E': 'S3'}, 'S3': {'isTerminatingState': True, 'A': 'S1', 'B': 'S0', 'E': 'S2'}}


In [10]:
def getClosure(startState, states):
    closure = []
    closure.append(startState)
    isAddedToClosure=True
    while isAddedToClosure:
        isAddedToClosure=False
        for item in closure:
            for key , value in states[item].items():
                if key== ' ' and value not in closure:
                    closure.append(value)
                    isAddedToClosure=True;
    return closure

In [11]:
print (getClosure('S0',states))

['S0', 'S2', 'S1']


In [12]:
class DFA:
    def __init__(self):
        self.startState = None
        self.acceptStates = []
        self.States = []
        self.transitions = {}

DFA = DFA()
DFA.startState = getClosure(startState, states)

st = []
st.append(DFA.startState)

while st:
    transMap = {}
    mainState = st.pop()
    mainState.sort()
    if mainState not in DFA.States:
        DFA.States.append(mainState)
    isAccept = False
    for item in mainState:
        for key, value in states[item].items():
            if key == "isTerminatingState" and value == True:
                isAccept = True
            if key != "isTerminatingState" and key != ' ':
                if key not in transMap:
                    transMap[key] = []
                transMap[key].append(value)
                value2 = getClosure(value, states)
                for Closureitem in value2:
                    transMap[key].append(Closureitem)
    if isAccept == True and mainState not in DFA.acceptStates:
        DFA.acceptStates.append(mainState)
    # Remove duplicates from each list in transMap
    for key in transMap:
        transMap[key] = list(set(transMap[key]))
        transMap[key].sort()
        if transMap[key] not in DFA.States:
            DFA.States.append(transMap[key])
            st.append(transMap[key])
        if tuple(mainState) not in DFA.transitions:
                DFA.transitions[tuple(mainState)] = {}
            
        DFA.transitions[tuple(mainState)][key] = transMap[key]    

In [13]:
anotherMap = {}
count = 0
for item in DFA.States:
    anotherMap[tuple(item)] = 'S' + str(count)
    count += 1
DFA.startState = anotherMap[tuple(DFA.startState)]
# Convert DFA.States
for i in range(len(DFA.States)):
    DFA.States[i] = anotherMap[tuple(DFA.States[i])]

# Convert DFA.acceptStates
for i in range(len(DFA.acceptStates)):
    DFA.acceptStates[i] = anotherMap[tuple(DFA.acceptStates[i])]

# Convert DFA.transitions
new_transitions = {}
for key, value in DFA.transitions.items():
    new_key = anotherMap[tuple(key)]
    new_value = {k: anotherMap[tuple(v)] for k, v in value.items()}
    new_transitions[new_key] = new_value
DFA.transitions = new_transitions

In [14]:
#convert DFA to JSON
import json

# Create a dictionary that represents the DFA
dfa_dict = {
    "startingState": DFA.startState,
}

# Add states to the dictionary
for state in DFA.States:
    dfa_dict[state] = {
        "isTerminatingState": state in DFA.acceptStates,
    }
    # Add transitions for each state
    if state in DFA.transitions:
        for symbol, nextState in DFA.transitions[state].items():
            dfa_dict[state][symbol] = nextState

# Convert the dictionary to a JSON string
dfa_json = json.dumps(dfa_dict, indent=4)

print(dfa_json)
   

{
    "startingState": "S0",
    "S0": {
        "isTerminatingState": false,
        "A": "S1",
        "B": "S0",
        "E": "S2"
    },
    "S1": {
        "isTerminatingState": false,
        "A": "S1",
        "B": "S1",
        "E": "S0"
    },
    "S2": {
        "isTerminatingState": true,
        "A": "S1",
        "B": "S0",
        "E": "S2"
    }
}
